In [32]:
import os
import rasterio
import numpy as np
import geopandas as gpd

from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon, box


class SerialTimer():
    
    def __init__(self,
                 aoi,
                 raster_dir='/home/volume2/Austria/Timeseries/',
                 polygon_file='/home/volume2/Austria/GroundTruth/invekos_schlaege_polygon.shp',
                 out_dir='/home/mlamarre/Documents/FieldTimeSeries',
                 time_stamps=np.arange(1,61),
                 dimensions=['BS.VH', 'BS.VV']):
        
        self.raster_dir = raster_dir
        self.polygon_file = polygon_file
        self.out_dir = out_dir
        self.aoi = aoi
        self.time_stamps = time_stamps
        self.dimensions = dimensions
        
    def readPolygons(self):
        if os.path.isfile(self.polygon_file):
            self.polygons = gpd.read_file(self.polygon_file)
        else:
            print('ERROR: wrong shape file')
            
    def readRaster(self, time_stamps, dimension):
        filename = self.raster_dir

In [33]:
s = SerialTimer(2)

In [34]:
s.readPolygons()

ERROR: wrong shape file
